# Load Data

In [1]:
from pyearth import Earth
from DLtools.Data_preprocess import series_to_supervised,load_data
from DLtools.Data import load_data

import numpy as np

In [2]:
# r='data/instant_data/rain_small.csv'
# w='data/instant_data/water_small.csv'
# #r = "data/instant_data/rain.csv"
# #w = 'data/instant_data/water.csv'
# rw = load_data(r,w)
# df =rw.df.resample('d').mean()
loaddata = load_data()

df_d = loaddata.daily()
df_h = loaddata.hourly()


  2%|▏         | 3/154 [00:00<00:05, 28.01it/s]START LOADING DATA 2012-2020(July)

rainfall (daily)....
356
Working on rain file.....
  0%|          | 0/113 [00:00<?, ?it/s]
Before del col are... 154
After... 123

water lv (hourly)....
149
Working on water file.....
100%|██████████| 113/113 [03:54<00:00,  2.07s/it]

Before del col are... 113
  0%|          | 0/1812 [00:00<?, ?it/s]After... 94

weather lv (hourly)....
355
working on Weather year 2012
100%|██████████| 1842/1842 [00:09<00:00, 204.59it/s]
(11987808,) temp
(11987808,) press
(11987808,) humid
(11987808,) solar
(11987808,) rain1h

Before del col are... 990
After... 252

Dam (daily)....
working on column: WaterVol
working on column: Useable_WaterVol
working on column: Input_WaterVol
working on column: Output_WaterVol

Before del col are... 20
After... 20
==========TOTAL FILE==========
Rain.............Water.........Weather...........Dam
(2769, 123) (398736, 94) (66456, 252) (2745, 20)


In [4]:
TARGET = 'CPY015_wl'

## Period 2013-2015 ##
data = df_d['2013-01-01':'2017-12-31'].interpolate(limit=360).fillna(0)
data = data.astype('float32')
X = data.drop([TARGET],axis=1)
xlabels = list(X.columns)


# REPLACE NAN WITH 0
X = X.fillna(0).values
y = np.log(data[TARGET].fillna(0).values)
where_are_NaNs = np.isnan(y)
y[where_are_NaNs] = 0


# MARS feature selection (from all stations)

In [5]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #   max_terms=10,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.327047  1      0.327  0.000  0.000  
1     0       150  730   0.153264  3      0.154  0.531  0.529  
2     0       164  1285  0.135050  5      0.137  0.587  0.582  
3     0       128  871   0.127577  7      0.130  0.610  0.603  
4     0       145  930   0.121213  9      0.124  0.629  0.621  
5     0       212  796   0.118781  11     0.122  0.637  0.627  
6     0       164  1380  0.116942  13     0.121  0.642  0.630  
7     0       164  1157  0.112945  15     0.118  0.655  0.641  
8     0       164  163   0.110952  17     0.116  0.661  0.645  
9     0       150  544   0.109757  19     0.116  0.664  0.647  
10    0       34   486   0.108567  21     0.115  0.668  0.649  
11    0       150  536   0.107647  23     0.115  0.671  0.650  
12    0       128

Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [5]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [11]:
print(model.summary_feature_importances(sort_by='rss')[:2700])

                               rss    gcv    nb_subsets
CPY014_wl                      0.54   0.61   0.06          
GLF001_wl                      0.23   0.25   0.08          
CPY012_wl                      0.05   0.04   0.08          
CPY010_wl                      0.02   0.03   0.02          
BKK007_wl                      0.02   0.01   0.05          
NAN013_solar                   0.02   0.01   0.06          
BKK018_wl                      0.01   0.01   0.02          
DNP022_wl                      0.01   0.01   0.02          
CPY009_temp                    0.01   0.00   0.02          
PIN005_solar                   0.01   0.00   0.02          
DNP032_solar                   0.01   0.00   0.02          
DIV003_rain                    0.00   0.00   0.03          
DNP055_solar                   0.00   0.00   0.02          
DIV004_rain                    0.00   0.00   0.02          
PAS007_wl                      0.00   0.00   0.02          
DNP032_rain                    0.00   0.00  

# MARS feature selection (Times interval perspective)

In [7]:
n_timelag=7

reframed_data = series_to_supervised(X_in.iloc[:20,:], n_in= 3,n_out=1)
# not work
X_in.shape,reframed_data.shape

((731, 35), (0, 140))

In [8]:
def to_sub(df,n_in=5,n_out=3):
    df_in = df.shift(n_in)
    df_out = df.shift(-n_out)
    return df_in

In [9]:
test = X_in.iloc[:,:5]

print(test.head())
print(to_sub(test).head())

            CHM002_r  CHM004_r  CHM006_r  CPY004_r  CPY009_r
date                                                        
2013-01-01       0.0       NaN       NaN       0.0       0.0
2013-01-02       0.0       NaN       NaN       0.0       0.0
2013-01-03       0.0       NaN       NaN       0.0       0.0
2013-01-04       0.0       NaN       NaN       0.0       0.0
2013-01-05       0.0       NaN       NaN       0.0       0.0
            CHM002_r  CHM004_r  CHM006_r  CPY004_r  CPY009_r
date                                                        
2013-01-01       NaN       NaN       NaN       NaN       NaN
2013-01-02       NaN       NaN       NaN       NaN       NaN
2013-01-03       NaN       NaN       NaN       NaN       NaN
2013-01-04       NaN       NaN       NaN       NaN       NaN
2013-01-05       NaN       NaN       NaN       NaN       NaN
